In [ ]:
import glob
import os

from astropy.io import fits
import numpy as np
import numpy.ma as ma
import pandas as pd

In [ ]:
working_directory = '/grp/hst/wfc3u/final_postflash_2021/biyearly_reference/'
today = '08-20-2021'
postflash_data = pd.read_pickle('/grp/hst/wfc3u/final_postflash_2021/July_2021_all_postflash_flc.pkl')
print(postflash_data)

In [ ]:
fullframe_pf = postflash_data.loc[(postflash_data['subarray'] == False)] 
fullframe_pf_A = postflash_data.loc[(postflash_data['subarray'] == False) & (postflash_data['shutter'] == 'A') & (postflash_data['flash_cur'] == 'MED') & (postflash_data['flash_dur'] == 100.0)] 
fullframe_pf_B = postflash_data.loc[(postflash_data['subarray'] == False) & (postflash_data['shutter'] == 'B') & (postflash_data['flash_cur'] == 'MED') & (postflash_data['flash_dur'] == 100.0)] 

In [ ]:
def stack(list_of_files,out_file,error_file):
    hdr = fits.getheader(list_of_files[0], 1)
    nx = hdr['NAXIS1']
    ny = hdr['NAXIS2']
    nf = len(list_of_files)
    # Setting up the empty data, rms, and error arrays and getting the data
    data_array_1 = np.empty((nf, ny, nx), dtype=float)
    data_array_2 = np.empty((nf, ny, nx), dtype=float)
    set_data=fits.getdata(list_of_files[0], 1)
    rms_1 = np.zeros(len(list_of_files), dtype=float)
    rms_2 = np.zeros(len(list_of_files), dtype=float)
    error_array_1 = np.empty((nf, ny, nx), dtype=float)
    error_array_2 = np.empty((nf, ny, nx), dtype=float)
    total_error_1 = np.zeros_like(set_data, dtype=float)
    total_error_2 = np.zeros_like(set_data, dtype=float)
    for i , f in enumerate(list_of_files):
        data_1=fits.getdata(f, 1)
        data_2=fits.getdata(f, 4)
        error_1=fits.getdata(f, 2)
        error_2=fits.getdata(f, 5)
        DQ_1=fits.getdata(f, 3)
        DQ_2=fits.getdata(f, 6)
        #data_1=data1
        #data_2=data2
        mask_1=np.zeros_like(data_1, dtype=bool)
        mask_2=np.zeros_like(data_2, dtype=bool)
        mask_1[DQ_1>=2**13]= True
        mask_2[DQ_2>=2**13]= True
        error_1[(0<DQ_1 & (DQ_1<2**13))]=0.00001
        error_2[(0<DQ_2& (DQ_2<2**13))]=0.00001
        error_1_sq=error_1**2
        error_2_sq=error_2**2
        masked_data_1= ma.array(data=data_1, mask=mask_1)
        masked_data_2= ma.array(data=data_2, mask=mask_2)
        data_array_1[i, :, :] = masked_data_1
        rms_1[i] = masked_data_1.std()
        data_array_2[i, :, :] = masked_data_2
        rms_2[i] = masked_data_2.std()
        total_error_1=total_error_1+(error_1_sq)
        total_error_2=total_error_2+(error_2_sq)
    sr_total_error_1=np.sqrt(total_error_1)
    sr_total_error_2=np.sqrt(total_error_2)
    fin_error_1=(sr_total_error_1/(float(len(list_of_files))))
    fin_error_2=(sr_total_error_2/(float(len(list_of_files))))
    image_median_1 = np.median(data_array_1, axis=0)
    image_median_2 = np.median(data_array_2, axis=0)
    new_hdul = fits.HDUList()
    new_hdul.append(fits.ImageHDU(image_median_1))
    new_hdul.append(fits.ImageHDU(image_median_2))
    new_hdul.writeto(out_file, overwrite=True)
    #error
    new_hdul = fits.HDUList()
    new_hdul.append(fits.ImageHDU(fin_error_1))
    new_hdul.append(fits.ImageHDU(fin_error_2))
    new_hdul.writeto(error_file,overwrite=True)

In [ ]:
fullframe_pf_A_2012_2016 = fullframe_pf_A[(fullframe_pf_A['datetime'] > '2012-01-01 00:00:00') & (fullframe_pf_A['datetime'] < '2017-01-01 00:00:00')]
paths_2012_2016_A = fullframe_pf_A_2012_2016.path.tolist()
print(len(paths_2012_2016))
outfile_2012_2016_A= '{}2012_2016_fullframe_A_flc_stack_{}.fits'.format(working_directory, today)
error_outfile_2012_2016_A = '{}2012_2016_fullframe_A_flc_error_stack_{}.fits'.format(working_directory, today)

In [ ]:
fullframe_pf_B_2012_2016 = fullframe_pf_B[(fullframe_pf_B['datetime'] > '2012-01-01 00:00:00') & (fullframe_pf_B['datetime'] < '2017-01-01 00:00:00')]
paths_2012_2016_B = fullframe_pf_B_2012_2016.path.tolist()
print(len(paths_2012_2016_B))
outfile_2012_2016_B = '{}2012_2016_fullframe_B_flc_stack_{}.fits'.format(working_directory, today)
error_outfile_2012_2016_B = '{}2012_2016_fullframe_B_flc_error_stack_{}.fits'.format(working_directory, today)

In [ ]:
stack(paths_2012_2016_A, outfile_2012_2016_A, error_outfile_2012_2016_A)

stack(paths_2012_2016_B, outfile_2012_2016_B, error_outfile_2012_2016_B)

In [ ]:
fullframe_pf_A_2017_2021 = fullframe_pf_A[(fullframe_pf_A['datetime'] > '2017-01-01 00:00:00') & (fullframe_pf_A['datetime'] < '2022-01-01 00:00:00')]
paths_2017_2021_A = fullframe_pf_A_2017_2021.path.tolist()
print(len(paths_2017_2021_A))
outfile_2017_2021_A = '{}2017_2021_fullframe_A_flc_stack_{}.fits'.format(working_directory, today)
error_outfile_2017_2021_A = '{}2017_2021_fullframe_B_flc_error_stack_{}.fits'.format(working_directory, today)

fullframe_pf_B_2017_2021 = fullframe_pf_B[(fullframe_pf_B['datetime'] > '2017-01-01 00:00:00') & (fullframe_pf_B['datetime'] < '2022-01-01 00:00:00')]
paths_2017_2021_B = fullframe_pf_B_2017_2021.path.tolist()
print(len(paths_2017_2021_B))
outfile_2017_2021_B = '{}2017_2021_fullframe_B_flc_stack_{}.fits'.format(working_directory, today)
error_outfile_2017_2021_B = '{}2017_2021_fullframe_B_flc_error_stack_{}.fits'.format(working_directory, today)

In [ ]:
stack(paths_2017_2021_A, outfile_2017_2021_A, error_outfile_2017_2021_A)

stack(paths_2017_2021_B, outfile_2017_2021_B, error_outfile_2017_2021_B)

In [ ]:
def change_permissions(path_to_files):
    """Change permissions of the output files and directories. 
        Code structure borrowed from cal_uvis_make_darks/FileIO.py. 
        Parameters
        ----------
            path_to_files : str
                Path to the directory that contains the files you want
                to update. 
    """
    
    os.chdir(path_to_files)
    all_directories = glob.glob('*')

    for directory in all_directories:
        try:
            os.chmod(directory, 0o775)
        except: 
            print('Could not change permissions.')
        for root, subdirs, files in os.walk(directory):
            for subdir in subdirs:
                try:
                    os.chmod(os.path.join(root, subdir), 0o775)
                except: 
                    print('Could not change permissions.')
            for name in files:
                try:
                    os.chmod(os.path.join(root, name), 0o775)
                except: 
                    print('Could not change permissions.')

In [ ]:
change_permissions(working_directory)